# Entrega 1, Grupo 02 - Arboles de decisión

- Santiago Alaniz, 5082647-6, santiago.alaniz@fing.edu.uy
- Bruno De Simone, 4914555-0, bruno.de.simone@fing.edu.uy
- Maria Usuca, , maria.usuca@fing.edu.uy



## 1. Objetivo

Implementar una variante de `ID3` para construir un modelo que explique la deserción de estudiantes en la universidad.
 
Se pide:

- **(a)** Implementar una variante del algoritmo `ID3` agregandole los siguientes *hiperparametros*:
    - **i)** `min_samples_split`: cantidad mínima de ejemplos para generar un nuevo nodo; en caso de que no se llegue a la cantidad requerida, se debe formar una hoja.
    - **ii)** `min_split_gain`: ganancia mínima requerida para partir por un atributo; si ningún atributo llega a ese valor, se debe formar una hoja.
- **(b)** Utilizar el algoritmo implementado en **(a)** para construir un arbol de decision, evaluar resultados utilizando el dataset provisto.
- **(c)** Discuta como afecta la variacion de los hiperparametros con los modelos obtenidos.
- **(d)** Corra los algoritmos de `scikit-learn` DecisionTreeClassifier, RandomForestClassifer y compare los resultados.

El dataset que vamos a considerar (con su debido preprocesamiento) es *Predict students dropout and accademic success* con **36 atributos y mas de 4000 instancias.**

## 2. Diseño

El apartado de diseño engloba todas las decisiones que tomamos a la hora de cumplir con las subtareas planteadas en la seccion anterior. 

Podemos identificar las siguientes etapas:

- **Carga de datos y Particionamiento**: Inicializacion de los datos de los archivos CSV en un DataFrame de Pandas.
- **Pre-procesamiento de datos**: Transformaciones necesarias para que los datos puedan ser utilizados por el modelo.
- **Algoritmo**: Comentarios sobre la implementacion del algoritmo asi como las decisiones tomadas para su implementacion.

### 2.1 Carga de datos y particionamiento

En este apartado vamos a inicializar los datos siguiendo un esquema clasico de aprendizaje automático:

- **Carga de datos**: Cargamos los datos desde el fichero `csv` y los almacenamos en un `DataFrame` de `pandas`.
- **Particionamiento**: Particionamos los datos en dos conjuntos con `train_test_split` de `sklearn`.



In [138]:
import pandas as pd
from sklearn.model_selection import train_test_split

CSV_PATH = './assets/data.csv'
SEED_NUMBER = 42069
TRAIN_SIZE = 0.8
TEST_SIZE = 0.2

data = pd.read_csv(CSV_PATH, sep=';')
train, test = train_test_split(data, test_size= TEST_SIZE, train_size= TRAIN_SIZE, random_state= SEED_NUMBER)

print(f'[ data: {data.shape[0]}, train: {train.shape[0]}, test: {test.shape[0]} ]')
train.head()

[ data: 4424, train: 3539, test: 885 ]


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
3687,1,1,1,9147,1,1,132.0,1,19,38,...,0,5,6,0,0.000000,0,13.9,-0.3,0.79,Dropout
2457,1,1,1,9147,1,1,166.0,1,19,37,...,0,5,12,1,13.000000,0,7.6,2.6,0.32,Enrolled
1289,1,1,1,9500,1,1,110.0,1,19,19,...,0,8,9,7,13.771429,0,11.1,0.6,2.02,Graduate
307,1,39,1,9130,1,3,150.0,1,3,1,...,0,5,5,0,0.000000,0,12.7,3.7,-1.70,Dropout
2369,1,39,1,9130,1,1,133.1,1,1,19,...,0,5,10,3,13.333333,2,10.8,1.4,1.74,Dropout


# 2.2 Pre-procesamiento de los datos

Para el pre-procesado de los datos tomaremos en cuenta los siguientes puntos:

- Redefinicion de los valores del atributo objetivo `Target` para que sean 0 y 1.
- Discretizacion de todos los atributos continuos.
- Comentario sobre el resto de los valores (discretos)

#### 2.2.1 Redefinicion de los valores del atributo objetivo `Target`.

El atributo objetivo `Target` es un atributo categorico que indica el desenlace del estudiante en su vida académica. Este atributo tiene 3 posibles valores: 

- `Enrolled` (inscripto)
- `Dropout` (abandono)
- `Graduate` (graduado).

La idea es construir un modelo sobre la diserción de los estudiantes, por lo que se decide agrupar los valores `Enrolled` y `Graduate` en un solo valor. 

-  0 &rarr; `Dropout`
-  1 &rarr; `Enrolled` o `Graduate`

**Nota**: 
La siguiente redefinicion de atributos genera un desbalance en el atributo `Target`. De todas formas, continuaremos con el analisis.

In [139]:
for df in [train, test]:
    df['Target'] = df['Target'].apply(lambda x: 0 if x == 'Dropout' else 1)

train['Target'].value_counts()

Target
1    2382
0    1157
Name: count, dtype: int64

#### 2.2.2 Pre-procesamiento de atributos continuos.

La [discretizacion](https://en.wikipedia.org/wiki/Data_binning) provee un mecanismo para particionar valores continuos en un numero finito de valores discretos.

De los [36 atributos presentes](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success) en el dataset, estos son listados como continuos:

- `Previous qualification (grade)`
- `Admission grade`
- `Unemployment rate`
- `Inflation rate`
- `GDP`

Para discretizar estos atributos, utlizaremos el modulo `scikit-learn.preprocessing`. 

En particular, la clase [`KBinsDiscretizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html#sklearn.preprocessing.KBinsDiscretizer) con los siguientes parametros no predeterminados:

- `encode= 'ordinal'` (codificacion de los bins) devuelve un array de enteros indicando a que bin pertenece cada valor.
- `strategy='kmeans'` (estrategia de discretizacion) utiliza el algoritmo de [k-means](https://en.wikipedia.org/wiki/K-means_clustering) para determinar los bins. 

Finalmente, identificar estos atributos en el dataset es una tarea sensilla, ya que son los unicos del tipo `float64`.

***Nota***: 

Hay un error en la [documentacion de los datos](https://archive.ics.uci.edu/dataset/697/predict+students+dropout+and+academic+success), figuran como discretos dos campos representados con `float64`:

- `Curricular units 1st sem (grade)`
- `Curricular units 2nd sem (grade)` 

Decidimos discretizarlos de todas formas, ya que algunas de las entradas tienen valores no enteros.

In [137]:
from sklearn.preprocessing import KBinsDiscretizer

float64_cols = data.select_dtypes(include=['float64']).columns

for float64_col in float64_cols:
    float64_col_discretizer = KBinsDiscretizer(subsample=None, encode='ordinal', strategy='kmeans')
    train[[float64_col]] = float64_col_discretizer.fit_transform(train[[float64_col]]).astype(int)
    test[[float64_col]] = float64_col_discretizer.transform(test[[float64_col]]).astype(int)

train[float64_cols].head()

,Previous qualification (grade),Admission grade,Curricular units 1st sem (grade),Curricular units 2nd sem (grade),Unemployment rate,Inflation rate,GDP
3687,1,1,1,0,3,0,3
2457,4,3,1,2,0,3,2
1289,0,0,3,3,1,1,3
307,3,3,1,0,2,4,1
2369,1,1,4,2,1,2,3


## 2.3 Algoritmo

El algoritmo a desarrollar es `ID3` como se presento en el teorico, con la incorporacion de ciertos meta-parametros que buscan evitar el sobreajuste del modelo.

Para lograr este objetivo, se tuvo en consideracion las siguientes sub-tareas:

1. Definir y obtener las variables necesarias para implementar `ID3` (Mitchell, 97, p56).
2. `ID3_utils.py`: Un modulo con estructuras/funciones auxiliares para la implementacion de `ID3`.
3. Codigo del algoritmo `ID3`.
3. `G02DecisionTrees.ID3Classifier`: Un diseño modular del algoritmo inspirado en `sklearn`.

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from src.G02_algorithm import CustomID3Classifier
import matplotlib.pyplot as plt

X_train, y_train = train.drop(columns=['Target']), train['Target']

custom_clf = CustomID3Classifier(MIN_SAMPLES_SPLIT=0, MIN_SPLIT_GAIN=0)
custom_clf.fit(X_train, y_train)

In [ ]:
X_test, y_test = test.drop(columns=['Target']), test['Target']
predictions = custom_clf.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

conf_matrix = confusion_matrix(y_test, predictions)
disp = ConfusionMatrixDisplay(conf_matrix)
disp.plot(cmap=plt.cm.Blues, values_format='.2f')

plt.title("Confusion Matrix")
plt.show()



## 2.3 Evaluación
- Qué conjunto de métricas se utilizan para la evaluación de la solución y su definición
- Sobre qué conjunto(s) se realiza el entrenamiento, ajuste de la solución, evaluación, etc. Explicar cómo se construyen estos conjuntos.

## 3. Experimentación

- Presentar los distintos experimentos que se realizan y los resultados que se obtienen.

- La información de los resultados se presenta en tablas y en gráficos, de acuerdo a su naturaleza. Por ejemplo:

_En la gráfica 1, se observa el error cuadrático total del conjunto de entrenamiento a medida que pasan los juegos para el oponente X_


- Debe existir alguna instancia donde se compile la información relevante de los experimentos de forma de poder comparar fácilmente los distintos experimentos. Por ejemplo:

_En la tabla 1, se presentan los distintos resultados contra el jugador aleatorio, para los distintos valores de $\alpha$ elegidos. El mejor resultado se obtiene para $\alpha=0.05$, lo que prueba que la estrategia..._

<table>
  <tr>
    <th>$\alpha$</th>
    <th>...</th>
    <th>Turnos</th>
    <th>Error</th>
  </tr>
  <tr>
    <td>0.001</td>
    <td>...</td>
    <td>100</td>
    <td>0.991</td>
  </tr>    
  <tr>
    <td>0.005</td>
    <td>...</td>
    <td>100</td>
    <td>0.987</td>
  </tr>
  <tr style="font-weight:bold">
    <td>0.05</td>
    <td>...</td>
    <td>100</td>
    <td>0.329</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>...</td>
    <td>100</td>
    <td>0.564</td>
  </tr>    
    <caption>Tabla 1 - Entrenamiento del jugador X para distintos valores de $\alpha$</caption>
</table>


## 4. Conclusión

Una breve conclusión del trabajo realizado. Por ejemplo: 
- ¿cuándo se dieron los mejores resultados del jugador?
- ¿encuentra alguna relación con los parámetros / oponentes/ atributos elegidos?
- ¿cómo mejoraría los resultados?